# Creating the badges for the Jupyter Community Workshop: *Building the Jupyter Community in MSK Imaging Research*

by [Serena Bonaretti](https://sbonaretti.github.io/)

License for narrative: CC-BY  
License for code: GNU_GPL 3.0

---

- The code in this notebook creates badges of dimensions 95x65mm (3.74x2.55inches) in an A4 paper sheet  
- Each badge is composed of a front and a back, and it will be folded in half vertically
  - The front contains name of the workshop, name of the participant, and date and place 
  - The back contains food choices and requirements
- For each page, there are three badges, stacked one below the other
- See examples of badges in the folder *badges to print*   
- The code is adapted from [this repository](https://github.com/vrooje/conference-badges) 

- Notes for printing: 
  - Save files in .png, not .pdf because it looses resolution (feature of the package PIL?)  
  - The resolution is not optimal when printing. The reason is that there is a resolution loss every time one copies/pastes in PIL (copying/pasting happens in this code to create the page containing the badges - this can be improved)

- Participants' data are in the file `participants.csv`. The file is organized as follow: 
  - First name (column A): First name and middle name (if present) of participant
  - Last name (column B): Last name(s) of participants
  - Lunch (columns D-F): Participants had to choose among meat/fish/vegetarian options for 3 days
  - Dinner on first night: Participants had to choose appetizer, main, dessert, and wine (optional)  
  - Dinner on second night: Fixed menu, so participants just confirmed presence
  - Food requirements   

--- 
Imports: 

In [1]:
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont

--- 
## Reading and tidying the participants' .csv file

- Read the tabular file containing the participants' data and simplify column names:  
  *Note:* The tabular file is the output of a Google form, where each question became a column name 

In [2]:
filename = "./participants.csv" 
df = pd.read_csv(filename)

In [3]:
# change some column names 
df.rename(columns = {"Please, specify your food requirements"                   :"food_requirements",
                      "DAY 1 (Thursday, June 9) - What options would you like?" : "lunch_1",
                      "DAY 2 (Friday, June 10) - What options would you like?"  : "lunch_2",
                      "DAY 3 (Saturday, June 11) - What options would you like?": "lunch_3",
                      "Appetizer"               : "appetizer_1",
                      "Main course"             : "main_1",
                      "Dessert"                 : "dessert_1",
                      "Wine - prices per glass" : "wine_1",
                      "Dinner 2"                : "dinner_2"},
                      inplace = True)
df

,First name,Last name,lunch_1,lunch_2,lunch_3,appetizer_1,main_1,dessert_1,wine_1,dinner_2,food_requirements
0,Jane,Smith,Fish,Vegetarian,Fish,"Green asparagus, avocado, muhammara, egg, parm...","Sea bass with fresh vegetables, fennel, samphi...",Coupe sorbet ice cream with fresh fruit,"I will not drink wine, thanks!","Yes, i will be there!",None
1,Marco,Rossi,Fish,Meat,Meat,"Beef carpaccio with truffle, parmesan, pine nu...","Sea bass with fresh vegetables, fennel, samphi...",Tiramisu (various flavors),Merlot,"Thank you for the invitation, but I will not b...",Gluten-free
2,Jasmine,Aziz,"Thanks, but I will not be at lunch","Thanks, but I will not be at lunch","Thanks, but I will not be at lunch","Green asparagus, avocado, muhammara, egg, parm...","Flank steak with fresh vegetables, purple truf...",Tiramisu (various flavors),"I will not drink wine, thanks!","Yes, i will be there!",Dairy-free
3,Francisco,González,Vegetarian,Meat,Vegetarian,"Thank you for the invitation, but I will not b...","Thank you for the invitation, but I will not b...","Thank you for the invitation, but I will not b...","I will not drink wine, thanks!","Yes, i will be there!",Shellfish allergy


- For each participant, reformulate the choices with abbreviations:  
  *Note*: The code below can surely be optimized. The current status is due to historical reasons (i.e. changes of menus along the way)

In [4]:
# create a dictionary that will be transformed into a dataframe at the end of the cell
dictionary = {}
dictionary["First name"] = df["First name"].to_list()
dictionary["Last name"]  = df["Last name"].to_list()

# lists to be filled out in the for loop for each participant
lunches  = []
dinner_1 = []
wine_1   = []
dinner_2 = []
food_requirements = []

# for each participant
for i in range (0, df.shape[0]):
    
    # ----- lunches -----
    string = ""
    
    # lunch day 1
    # if the participant is not joining lunch, add a minus to the string (the bar is a divider - see the printed badge)
    if df.loc[i,"lunch_1"] == "Thanks, but I will not be at lunch":
        string += "- | "
    # if the participant is joining lunch, add their option to the sting
    else:
        string += df.loc[i,"lunch_1"] + " | "
    
    # lunch day 2
    if df.loc[i,"lunch_2"] == "Thanks, but I will not be at lunch":
        string += "- | "
    else:
        string += df.loc[i,"lunch_2"] + " | "
      
    # lunch day 3
    if df.loc[i,"lunch_3"] == "Thanks, but I will not be at lunch":
        string += "-"
    else:
        string += df.loc[i,"lunch_3"]
     
    # the somebody never joins 
    if string == "- | - | -":
        string = "-"
    
    lunches.append (string)

    
    # ----- dinner 1 -----
    string = ""
    
    # dinner 1 - appetizer
    # in the Google form, participants could choose among 3 options
    if df.loc[i,"appetizer_1"] == "Green asparagus, avocado, muhammara, egg, parmesan, pumpkin seeds":
        string += "Green asparagus | " 
    elif df.loc[i,"appetizer_1"] == "Beef carpaccio with truffle, parmesan, pine nuts, arugula":
        string += "Beef carpaccio | "
    elif df.loc[i,"appetizer_1"] == "Thank you for the invitation, but I will not be able to join":
        string += "-"

    # dinner 1 - main
    if df.loc[i,"main_1"] == "Sea bass with fresh vegetables, fennel, samphire, potatoes":
        string += "Sea bass | "
    elif df.loc[i,"main_1"] == "Flank steak with fresh vegetables, purple truffle potato, potatoes, and chimichurri":
        string += "Flank steak | "
    elif df.loc[i,"main_1"] == "Vegetarian option (To be determined)":
        string += "Vegetarian | "

    # dinner 1 - dessert
    if df.loc[i,"dessert_1"] == "Coupe sorbet ice cream with fresh fruit":
        string += "Sorbet"
    elif df.loc[i,"dessert_1"] == "Tiramisu (various flavors)":
        string += "Tiramisu"
        
    dinner_1.append(string)

    # dinner 1 - wine
    string = ""
    if df.loc[i,"wine_1"] == "I will not drink wine, thanks!":
        string += ""
    else:
        string = df.loc[i,"wine_1"]
    
    wine_1.append(string)
    
    
    # ----- dinner 2 ----- 
    if df.loc[i,"dinner_2"] == "Thank you for the invitation, but I will not be able to join":
        string = "-"
    else:
        string = "Table royal menu"
    dinner_2.append (string)

    
    # ----- food requirements -----
    string = ""
    
    if df.loc[i,"food_requirements"] == "None":
        string += "-"
    else:
        string += df.loc[i,"food_requirements"]
    
    food_requirements.append (string)

# add the 4 lists to the dictionary
dictionary["lunches"]  = lunches
dictionary["dinner_1"] = dinner_1
dictionary["wine_1"]   = wine_1
dictionary["dinner_2"] = dinner_2 
dictionary["food_requirements"] = food_requirements 
    
# transform the dictionary in a pandas dataframe
df = pd.DataFrame (dictionary)
df

,First name,Last name,lunches,dinner_1,wine_1,dinner_2,food_requirements
0,Jane,Smith,Fish | Vegetarian | Fish,Green asparagus | Sea bass | Sorbet,,Table royal menu,-
1,Marco,Rossi,Fish | Meat | Meat,Beef carpaccio | Sea bass | Tiramisu,Merlot,-,Gluten-free
2,Jasmine,Aziz,-,Green asparagus | Flank steak | Tiramisu,,Table royal menu,Dairy-free
3,Francisco,González,Vegetarian | Meat | Vegetarian,-,,Table royal menu,Shellfish allergy


---
## Creating the badges

- Constants: 

In [5]:
# multiplication factor for dpi and fonts (not sure why but it works!)
f = 3
# image dpi - resolution of image (dots per inch) 
DPI = 300 * f
# multiplication factor for fonts and spacing - to be changed proportionally according to DPI
FACTOR = 4 * f


# --- Dimensions ---

# A4 paper size in inches, then transformed in pixels
paper_size_in_inches = (8.27, 11.69)
paper_size_in_pixels = (int(DPI*paper_size_in_inches[0]), int(DPI*paper_size_in_inches[1]))
A4_HEIGHT = paper_size_in_pixels[1]
A4_WIDTH  = paper_size_in_pixels[0]

# badge in inches, then transformed in pixels
badge_size_in_inches = [3.74, 2.55]  
badge_size_in_pixels = [int(DPI*badge_size_in_inches[0]), int(DPI*badge_size_in_inches[1])]
BADGE_HEIGHT = badge_size_in_pixels[1]
BADGE_WIDTH  = badge_size_in_pixels[0]

# pad between lines for back of the badge
PAD = 14 * FACTOR 
# number of horizontal parts the back of the badge is composed of 
N_PARTS = 8


# --- Fonts ---

# font size for:
# - background (BG), that is header and footer  (badge front)
FONT_SIZES_BG    = 14 * FACTOR
# - first and last name (badge front)
FONT_SIZES_NAME  =  2 * FONT_SIZES_BG 
# - headers (badge back)
FONT_SIZES_DAYS  = 11 * FACTOR
# - meals (body in badge back) 
FONT_SIZES_MEALS =      FONT_SIZES_DAYS

# font used is Raleway, downloaded from Google font. 
# It can be substituted with any font already installed on one's computer
FONT_BG    = ImageFont.truetype('/Library/Fonts/Raleway-Light.ttf', FONT_SIZES_BG)
FONT_NAME  = ImageFont.truetype('/Library/Fonts/Raleway-Bold.ttf', FONT_SIZES_NAME)
FONT_DAYS  = ImageFont.truetype('/Library/Fonts/Raleway-Light.ttf', FONT_SIZES_DAYS)
FONT_MEALS = ImageFont.truetype('/Library/Fonts/Raleway-Bold.ttf', FONT_SIZES_MEALS)

# text colors (orange for names, black for everything else)
TEXT_COLOR_ORANGE = (243,118,38) 
TEXT_COLOR_BLACK  = (1, 1, 1)

# --- Saving ---
FILE_FORMAT   = ".png" # do not save in .pdf because it blurs out a lot
output_folder = "./badges_to_print/"

- Creating and printing the badge:  
  Steps:   
  1. Create a bounding box that will contain 3 badges, stacked one below the other  
  2. For each participant:    
     a. Create the front of the badge (with name and event details) - it will be on the left   
     b. Create the back of the badge (with menus) - it will be on the right  
     c. Paste front and back of the badge to its row in the bounding box  
  3. Every 3 participants:   
     a. Create the file name and save   
     b. Exception: if the last bounding box does not contain 3 badges, add white squares  

In [6]:
# create the badge canvas
badge = Image.new('RGB', (BADGE_WIDTH*2, BADGE_HEIGHT*3+2)) # +2 for the border (see next line)
# adding a black background to create the black line around badges. 
# The black background will be overlapped by white badges (except the border)
badge = ImageOps.expand(badge,border=1,fill='black') 

# for each participant
for i in range (0, df.shape[0]):
    
    
    # --- badge front --- 
    
    # make a white rectangle with background
    img_front = Image.new('RGB', (BADGE_WIDTH, BADGE_HEIGHT), color = 'white')

    # add top text and bottom
    d = ImageDraw.Draw(img_front)
    d.fontmode = "L"
    d.text((BADGE_WIDTH/2,PAD), "JC|MSK workshop", 
           font=FONT_BG, fill=TEXT_COLOR_BLACK, anchor="mm")
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT-PAD), "9-11 June 2022, Maastricht (NL)", 
           font=FONT_BG, fill=TEXT_COLOR_BLACK, anchor="mm")

    # add name
    first_name = df.loc[i, "First name"]
    last_name  = df.loc[i, "Last name"]

    # make sure that each word is in a new line and centered
    string = first_name + " " + last_name
    words = string.split()

    # print centered name composed of two words (e.g. first_name last_name)
    if len(words) == 2:
        # first word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2-PAD), words[0], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")
        # second word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2+PAD), words[1], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")

    # print centered name composed of three words (e.g. first_name middle_name last_name)
    elif len(words) == 3:
        # first word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2-FONT_SIZES_NAME/2-PAD), words[0], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")
        # second word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2), words[1], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")
        # third word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2+FONT_SIZES_NAME/2+PAD), words[2], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")  

    # print centered name composed of four words (e.g. first_name middle_name first_last_name second_last_name)
    elif len(words) == 4:
        # first word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2-FONT_SIZES_NAME - PAD), words[0], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")
        # second word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2-PAD), words[1], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")
        # third word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2+PAD), words[2], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")   
        # third word
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/2+FONT_SIZES_NAME + PAD), words[3], 
           font=FONT_NAME, fill=TEXT_COLOR_ORANGE, anchor="mm")   


    # --- badge back --- 

    # make a white rectangle with background
    img_back = Image.new('RGB', (BADGE_WIDTH, BADGE_HEIGHT), color = 'white')
    # add a black border
    # img_back = ImageOps.expand(img_back,border=1,fill='black')

    # add fixed text
    d = ImageDraw.Draw(img_back)
    d.fontmode = "L"
   
    # text positions 
    # see: https://pillow.readthedocs.io/en/stable/handbook/text-anchors.html
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*1-PAD), "Lunch (Thu | Fri | Sat)", 
           font=FONT_DAYS, fill=TEXT_COLOR_BLACK, anchor="mt")
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*3-PAD), "Dinner (Thu)", 
           font=FONT_DAYS, fill=TEXT_COLOR_BLACK, anchor="mt")
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*5-PAD), "Dinner (Fri)", 
           font=FONT_DAYS, fill=TEXT_COLOR_BLACK, anchor="mt")
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*7-PAD), "Food requirements", 
           font=FONT_DAYS, fill=TEXT_COLOR_BLACK, anchor="mt")
    
    # text contents
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*1), df.loc[i,"lunches"], 
           font=FONT_MEALS, fill=TEXT_COLOR_BLACK, anchor="mt")
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*3), df.loc[i,"dinner_1"], 
           font=FONT_MEALS, fill=TEXT_COLOR_BLACK, anchor="mt")
    if df.loc[i,"wine_1"] != "":
        d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*3+FONT_SIZES_MEALS), df.loc[i,"wine_1"], 
           font=FONT_MEALS, fill=TEXT_COLOR_BLACK, anchor="mt")
        
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*5), df.loc[i,"dinner_2"], 
           font=FONT_MEALS, fill=TEXT_COLOR_BLACK, anchor="mt")
    
    d.text((BADGE_WIDTH/2,BADGE_HEIGHT/N_PARTS*7), df.loc[i,"food_requirements"], 
           font=FONT_MEALS, fill=TEXT_COLOR_BLACK, anchor="mt")
    
    # concatenate images and save
    # paste first badge on badges bounding box
    if i%3 == 0:
        badge.paste(img_front, (1, 1)) # 1 to keep into account the border
        badge.paste(img_back, (BADGE_WIDTH+1, 1))
    # paste second badge on badges bounding box
    elif i%3 == 1:
        badge.paste(img_front, (1, BADGE_HEIGHT+2)) # 1 to keep into account the border
        badge.paste(img_back, (BADGE_WIDTH+1, BADGE_HEIGHT+2))
    # paste third badge on badges bounding box
    elif i%3 == 2:
        badge.paste(img_front, (1, BADGE_HEIGHT*2+3)) # 1 to keep into account the border
        badge.paste(img_back, (BADGE_WIDTH+1, BADGE_HEIGHT*2+3))
    
        # filename for save
        file_name = output_folder + "badge_" + str(i-2) + "_" + str(i) + FILE_FORMAT 
        # add badges to A4 paper
        paper = Image.new('RGB', (A4_WIDTH, A4_HEIGHT), color = 'white')

        paper.paste(badge, (round((A4_WIDTH-badge.size[0])/2), round((A4_HEIGHT-badge.size[1])/2)))
        # save
        paper.save(file_name, quality=DPI)
    
    # save the last page
    # if there there are not 3 badges in the last page
    if i == df.shape[0]-1 and i%3 != 2:
        # if there is only 1 badge
        if i%3 == 0: 
            # add white squares (otherwise need to print a lot of black)
            img_white = Image.new('RGB', (BADGE_WIDTH, BADGE_HEIGHT), color = 'white')
            badge.paste(img_white, (1, BADGE_HEIGHT+2)) # 1 to keep into account the border
            badge.paste(img_white, (BADGE_WIDTH+1, BADGE_HEIGHT+2))
            badge.paste(img_white, (1, BADGE_HEIGHT*2+3)) # 1 to keep into account the border
            badge.paste(img_white, (BADGE_WIDTH+1, BADGE_HEIGHT*2+3))
            # filename for save
            file_name = output_folder + "badge_" + str(i) + FILE_FORMAT
        # if there are only 2 badges
        elif i%3 == 1:
            # add white squares (otherwise need to print a lot of black)
            img_white = Image.new('RGB', (BADGE_WIDTH, BADGE_HEIGHT), color = 'white')
            badge.paste(img_white, (1, BADGE_HEIGHT*2+3)) # 1 to keep into account the border
            badge.paste(img_white, (BADGE_WIDTH+1, BADGE_HEIGHT*2+3))
            # filename for save
            file_name = output_folder + "badge_"  + str(i-1) + "_" +  str(i) + FILE_FORMAT
    
        # add badges to A4 paper
        paper = Image.new('RGB', (A4_WIDTH, A4_HEIGHT), color = 'white')
        paper.paste(badge, (round((A4_WIDTH-badge.size[0])/2), round((A4_HEIGHT-badge.size[1])/2)))
        
        # save
        paper.save(file_name, quality=DPI)
paper   

--- 
## Dependencies

In [7]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2022-06-23T16:06:02.612687+02:00

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 21.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

PIL   : 8.4.0
pandas: 1.3.4

